## Análisis de datos con Pandas (II)

### Consultas y agrupación de datos

Con Pandas se pueden realizar consultas de datos (querys) similares a las de SQL. Para realizar estas consultas se debe indicar el dataframe que contiene los datos a analizar (origen de los datos - FROM), la serie o series (recuerde que una serie es una columna indexada) que requerimos como respuesta (SELECT) y además una condición que se encargue de filtrar los datos (WHERE).

Se utilizará el dataframe myEnergy_df para los ejemplos.

In [ ]:
import pandas as pd
#direccion = r"C:\Users\juanf\OneDrive\Escritorio\Python\Building_Energy_Benchmarking_2018.csv"
energy_df = pd.read_csv("Building_Energy_Benchmarking_2018.csv", sep=",")
myEnergy_df = energy_df[['OSEBuildingID', 'YearBuilt', 'NumberofFloors',
                         'ENERGYSTARScore','Electricity(kBtu)', 
                         'GHGEmissionsIntensity']]
myEnergy_df = myEnergy_df.rename(columns={'OSEBuildingID': "id",
                                      'YearBuilt': "yearBuilt",
                                      'NumberofFloors': "numberOfFloors",
                                      'ENERGYSTARScore': "energyStarScore",
                                      'Electricity(kBtu)': "electricity",
                                      'GHGEmissionsIntensity': "emissionsIntensity"
                                       })

myEnergy_df.set_index('id', inplace=True) 
myEnergy_df.head(10)

### Sintaxis para consultas

La sintaxis para consultar datos en pandas es:

       dataframe[ condicion ] [Columnas]
       
Con esta sintaxis se obtendrá como respuesta otro dataframe.

In [ ]:
#Consultar datos de forma condicional
# dataframe: myEnergy_df
# condicion: myEnergy_df['numberOfFloors'] == 15
# columnas: ["id", "yearBuilt", "numberOfFloors"]

myEnergy_df [ myEnergy_df['numberOfFloors'] == 15 ] [ ["yearBuilt", "numberOfFloors"] ]

Se puede complementar la sintaxis anterior agregando un método al final de la consulta para operar directamente los datos.

       dataframe[ condicion ] [Columnas].metodo(argumentos)

In [ ]:
# Se agregan metodos al final de la consulta
myEnergy_df[ myEnergy_df['numberOfFloors'] == 15 ] [ ["yearBuilt", "numberOfFloors"] ].sort_values(by="yearBuilt")

In [ ]:
# Se agregan metodos al final de la consulta
myEnergy_df[ myEnergy_df['numberOfFloors'] == 15 ][["yearBuilt", "numberOfFloors"]].count()
# min, max, count, mean, std, sum

In [ ]:
# las consultas las pueden separar en partes
a = myEnergy_df[ myEnergy_df['numberOfFloors'] == 15 ]
b = a[["yearBuilt", "numberOfFloors"]]
c = b.sort_values(by="yearBuilt")

print(c)

###  Operadores lógicos

Pandas cuenta con los operadores:

       and (&) - intersección: devuelve los datos que cumplan ambas condiciones
       or (|) - unión: devuelve los datos que cumplan al menos una de las dos condiciones
       not (~) - complemento: devuelve los datos que no cumplan con la condición 

In [ ]:
# el operador lógico "and" se escribe en Pandas como: & 
myEnergy_df[ (myEnergy_df["yearBuilt"] < 2015) & (myEnergy_df["energyStarScore"] > 99) ]

In [ ]:
#el operador lógico "or" se escribe en Pandas como: | 
myEnergy_df[ (myEnergy_df["yearBuilt"] == 2000) |  (myEnergy_df["yearBuilt"] == 2001) ]

Puede obtener los estadísticos de la tabla o columna que resulta de la consulta.

In [ ]:
myEnergy_df[(myEnergy_df["yearBuilt"] == 2000) | (myEnergy_df["yearBuilt"] == 2001)].mean()

### Agrupación de datos

Con la función "groupby" se pueden agrupar datos según una o varias columnas. Al usar esta función es obligatorio indicar un método que opere a los datos que están siendo agrupados.

Observe que al agrupar los datos, las columnas de agrupación toman el lugar de columna de identificadores.

In [ ]:
# Se puede agrupar los datos
myEnergy_df.groupby('yearBuilt').min()[["electricity", "emissionsIntensity"]]

In [ ]:
myEnergy_df.groupby(["yearBuilt", "numberOfFloors"]).mean()

La respuesta a una agrupación es un nuevo dataframe el cual puede continuar operándose.

In [ ]:
myEnergy_df.groupby(["yearBuilt"]).mean()["electricity"]

In [ ]:
myEnergy_df.groupby(["yearBuilt", "numberOfFloors"]).mean()["emissionsIntensity"]

### Operaciones con series

Una forma de operar entre las columnas indexadas es utilizar funciones lambda junto al método "apply".

In [ ]:
# la sintaxis de lambda es:
# lambda [argumentos]: [operacion]
nuevaColumna = myEnergy_df["electricity"].apply(lambda dato: dato/10**6)
nuevaColumna

In [ ]:
# (x*3.14 + 15)/(10^6)
nuevaColumna2 = myEnergy_df["electricity"].apply(lambda x: (x*3.14+15)/10**6 )
print (nuevaColumna2)

Las series generadas pueden añadirse al dataframe utilizando la notación de corchetes.

In [ ]:
# Para agregar la columna a la tabla
# nombreDelDataFrame["nombreDeLaColumnaNueva"] = datos
myEnergy_df["electricity(M)"] = nuevaColumna
myEnergy_df.head()

In [ ]:
myEnergy_df["electricity(2)"] = myEnergy_df["electricity"].apply(lambda dato: dato/10**6)
myEnergy_df.head()

Para elimar columnas se utiliza la palabra clave "del" mientras que para eliminar filas se utiliza la función "drop".

In [ ]:
# Eliminar una columna
del myEnergy_df["electricity(2)"]
myEnergy_df.head()

In [ ]:
# Para eliminar una fila
#dataframe.drop([id], axis=0)
myEnergy_df.drop([2,3], axis=0)

### Iterar a través de una tabla

Se puede iterar a través de las filas de una tabla generando un elemento iterable al utilizar el método "iterrrows".

In [ ]:
# Para recorrer fila por fila una tabla
# for [variable] in [elemento iterable]:
#      cuerpo

for index, row in myEnergy_df.iterrows():  
    if row["emissionsIntensity"] > 20:
        print(row)
        print(type(row))
        print(index)

In [ ]:
myEnergy_df[myEnergy_df["emissionsIntensity"]>20]

Puede iterar a través de la tabla y obtener datos para crear un nueva columna. Es una alternativa al uso de las funciones lambda.

In [ ]:
# indiceNuevo = ((emissionsIntensity * energyStarScore) / (electricity) *(10**5)
indiceNuevo = []
calificacion = []  # A B C D 

for index, row in myEnergy_df.iterrows():
    indicador = ((row["emissionsIntensity"]* row["energyStarScore"]/row["electricity"])*(10**5))
    indiceNuevo.append(indicador)
    
    if row["emissionsIntensity"] < 2.5:
        calificacion.append("A")
    elif row["emissionsIntensity"] < 5:
        calificacion.append("B")
    elif row["emissionsIntensity"] < 7.5:
        calificacion.append("C")
    else:
        calificacion.append("D")
                     
        
#print(indiceNuevo)

myEnergy_df["INDICADOR05"] = indiceNuevo
myEnergy_df["Calificacion"] = calificacion
myEnergy_df.head(3)

### Para practicar:

1. Se requiere conocer el promedio de energyStarScore para todos los edificios agrupados por yearBuilt.

2. Se requier conocer la desviación estandar de emisiones(emissionsIntensity) para todos los edificios que tienen la misma cantidad de pisos (numberOfFloors). 

3. Realice una consulta para obtener los datos de los edificios cuyo consumo de electricidad (electricity) sea menor que la media.

4. Realice una consulta para conocer el edificio o edificios que tienen la mayor puntuación de energyStarScore.

5. Realice una consulta para conocer el edificio o los edificios que tienen el menor consumo de electricidad (electricity).

### Visualización de datos

Pandas cuenta con funciones integradas de Matplotlib, por lo que se pueden graficar directamente las series o dataframes.

In [ ]:
myEnergy_df["yearBuilt"]

In [ ]:
# Histograma
import matplotlib.pyplot as plt
myEnergy_df["yearBuilt"].plot(kind="hist", edgecolor='black')
plt.title("Histograma - Año de construcción")
plt.xlabel("Año")
plt.ylabel("Frecuencia")
plt.show()

Esto no implica que no se puedan obtener gráficas por otros medios.

In [ ]:
# Histograma
import matplotlib.pyplot as plt

plt.hist(myEnergy_df["yearBuilt"], edgecolor='black')
plt.title("Histograma - Año de construcción")
plt.xlabel("Año")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
plt.hist(myEnergy_df["energyStarScore"], edgecolor='black')
plt.show()

La integración permite obtener gráficas utilizando pocas líneas de código.

In [ ]:
# Media del consumo energético según el número de pisos 
plt.figure(figsize=(5,14))
myEnergy_df.groupby(["yearBuilt"]).mean()["electricity"].plot(kind="barh")
plt.show()

### Para practicar

Grafique lo siguiente:

        1. Histograma del consumo de electricidad
        
        2. Histograma de la intensidad de emisiones de gases
        
        3. Gráfico de barras de la media del indicador energyStarScore por año entre el 2000 y el 2010  
        
        4. Gráfico de barras del valor máximo de intensidad de emisiones para edificios de 15, 16, 17, 18, 19 y 20 pisos.

### Bibliografía

Bibliografia de Pandas

https://pandas.pydata.org/docs/

https://github.com/pandas-dev/pandas


Visualización de datos con pandas

https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html


Bases de datos:

https://figshare.com

https://www.kaggle.com

https://www.kaggle.com/loveall/appliances-energy-prediction

https://www.data.gov

https://data.europa.eu/euodp/en/home

https://archive.ics.uci.edu/ml/datasets.php


Bases de datos en Google:

https://datasetsearch.research.google.com

Otros:

https://lionbridge.ai/datasets/the-50-best-free-datasets-for-machine-learning/
https://guides.library.cmu.edu/machine-learning/datasets

Extra: Gestión de bases de datos con python

https://realpython.com/tutorials/databases/